In [15]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from app.data_processing.utils import *

In [16]:
import pandas as pd

In [616]:
vg = pd.read_csv('../data/pesos_vegetales.csv', index_col = False)
ing = pd.read_csv('../data/ingredientes.csv',sep = '|')
recipes = pd.read_csv('../data/output.csv',sep='|')
bedca = pd.read_csv('../data/bedca.csv')

In [434]:
na_count = ing['Grams'].isna().groupby(ing['Recipe_id']).sum()
len(na_count[na_count == 0])


1910

In [619]:
def fill_grams(df):
    '''
    NaN weight values filler function.
        -The ingredients present in the weights dataset are substituted accordingly.
        -For the remaining ingredients: weight(ingredient) = mean(weight(ingredient))
    '''
    df['Index'] = df.index
    original_columns = list(df.columns)
    weights = df[df['Unidad']=='None']
    
    weights = pd.merge(weights, vg, how='inner', on='Ingrediente')

    weights['Grams'] = weights['Peso x Unidad']
    weights['Unidad'] = 'unidad'
    weights['Total_Grams'] = weights['Grams'].multiply(weights['Cantidad'], axis = 'index')
    
    
    weights = weights[original_columns]
    weights.set_index(weights['Index'], inplace=True)
    result = df.copy()
    result.loc[weights['Index'],:] = weights
    
    #añadir nº comensales
    result = pd.merge(result,recipes['Num_comensales'], how='left', left_on='Recipe_id', right_index=True)
    result['Num_comensales'] = result['Num_comensales'].fillna(1)
    result['Total_Grams'] = result['Total_Grams'].divide(result['Num_comensales']).round(5)
    
    result[['Grams','Total_Grams']] = result.groupby(['Ingrediente'])[['Grams','Total_Grams']].transform(lambda x: x.fillna(x.mean()).round())
    na_count = result['Total_Grams'].isna().groupby(result['Recipe_id']).sum()
    missing = list(na_count[na_count != 0].index)
    result = result[~result['Recipe_id'].isin(missing)]
    
    return result.drop(['Index'], axis=1)

In [620]:
result = fill_grams(ing)
result


,Recipe_id,Ingrediente,Cantidad,Unidad,Indice,Grams,Total_Grams,Num_comensales
0,74154,harina,10.0,None,482,21.0,116.0,1.0
1,74154,chorizo,500.0,gramos,181,1.0,500.0,1.0
2,74154,alubia,1.5,tazas,535,150.0,225.0,1.0
3,74154,queso,1.0,taza,77,150.0,150.0,1.0
4,74154,lechuga,1.0,rebanada,634,60.0,60.0,1.0
...,...,...,...,...,...,...,...,...
52379,5,plátano,1.0,unidad,721,80.0,13.0,6.0
52380,5,miel,2.0,cucharadas,761,10.0,3.0,6.0
52381,5,vainilla,1.0,cucharada,931,10.0,2.0,6.0
52382,5,leche,75.0,None,27,206.0,288.0,6.0


In [525]:
na_count = result['Total_Grams'].isna().groupby(result['Recipe_id']).sum()
na_count[na_count != 0].index


Int64Index([ 6954,  7641,  7912,  8240,  8886, 11030, 12962, 13324, 13392,
            13393, 15054, 15095, 16807, 17353, 19627, 19733, 21938, 21939,
            21940, 22210, 24754, 25004, 25667, 27612, 27713, 28029, 28100,
            28543, 28721, 29356, 29723, 30783, 31701, 32861, 37744, 41354,
            41483, 43865, 43934, 50140, 50141, 50195, 51681, 52029, 52840,
            52951, 58255, 59164, 60871, 72104, 74058],
           dtype='int64', name='Recipe_id')

In [559]:
recetas = pd.read_csv('../data/output.csv',sep='|')